# Empirical Asset Pricing via Machine Learning
## Interactive Exploration Notebook

This notebook provides interactive exploration of the Gu, Kelly, and Xiu (2020) replication.

**Sections:**
1. Data Overview
2. Model Performance Analysis
3. Feature Importance Deep Dive
4. Portfolio Analysis
5. Custom Analysis

In [ ]:
# Import libraries
import sys
from pathlib import Path

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))
from utils import *

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Directories
DATA_DIR = Path.cwd().parent / 'data'
RESULTS_DIR = Path.cwd().parent / 'results'

print("✓ Libraries imported successfully")

## 1. Data Overview

In [ ]:
# Load preprocessed data
train_df = pd.read_parquet(DATA_DIR / 'train_data.parquet')
test_df = pd.read_parquet(DATA_DIR / 'test_data.parquet')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"\nDate range:")
print(f"  Train: {train_df.index.get_level_values('date').min()} to {train_df.index.get_level_values('date').max()}")
print(f"  Test: {test_df.index.get_level_values('date').min()} to {test_df.index.get_level_values('date').max()}")

In [ ]:
# Load metadata
import json
with open(DATA_DIR / 'data_metadata.json', 'r') as f:
    metadata = json.load(f)

print(f"Target variable: {metadata['target_col']}")
print(f"Number of features: {metadata['n_features']}")
print(f"\nFirst 20 features:")
for i, feat in enumerate(metadata['feature_cols'][:20], 1):
    print(f"  {i}. {feat}")

In [ ]:
# Summary statistics
target_col = metadata['target_col']

print("Target variable statistics (excess returns):")
print(test_df[target_col].describe())

# Plot distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(test_df[target_col].dropna(), bins=100, alpha=0.7, edgecolor='black')
axes[0].set_title('Distribution of Excess Returns (Test Set)', fontweight='bold')
axes[0].set_xlabel('Excess Return')
axes[0].set_ylabel('Frequency')
axes[0].axvline(0, color='red', linestyle='--', alpha=0.5)

# QQ plot
from scipy import stats
stats.probplot(test_df[target_col].dropna(), dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot', fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Model Performance Analysis

In [ ]:
# Load predictions
benchmark_pred = pd.read_parquet(RESULTS_DIR / 'predictions' / 'benchmark_predictions.parquet')
gbrt_pred = pd.read_parquet(RESULTS_DIR / 'predictions' / 'gbrt_predictions.parquet')

print(f"Benchmark predictions: {benchmark_pred.shape}")
print(f"GBRT predictions: {gbrt_pred.shape}")

In [ ]:
# Calculate monthly R²
benchmark_r2 = calculate_monthly_r_squared(benchmark_pred.reset_index())
gbrt_r2 = calculate_monthly_r_squared(gbrt_pred.reset_index())

# Plot comparison
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(benchmark_r2.index, benchmark_r2.values * 100, label='OLS-3 Benchmark', 
        linewidth=2, alpha=0.7)
ax.plot(gbrt_r2.index, gbrt_r2.values * 100, label='GBRT', 
        linewidth=2, alpha=0.7)

# Add mean lines
ax.axhline(benchmark_r2.mean() * 100, color='blue', linestyle='--', alpha=0.5,
          label=f'Benchmark mean: {benchmark_r2.mean()*100:.3f}%')
ax.axhline(gbrt_r2.mean() * 100, color='orange', linestyle='--', alpha=0.5,
          label=f'GBRT mean: {gbrt_r2.mean()*100:.3f}%')

ax.set_title('Monthly Out-of-Sample R²', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('R² (%)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAverage monthly R²:")
print(f"  Benchmark: {benchmark_r2.mean()*100:.4f}% (Target: 0.16%)")
print(f"  GBRT: {gbrt_r2.mean()*100:.4f}% (Target: 0.33-0.40%)")
print(f"  Improvement: {(gbrt_r2.mean() / benchmark_r2.mean() - 1)*100:.1f}%")

In [ ]:
# Scatter plot: Predicted vs Actual
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Sample for visualization (5000 random points)
sample_size = 5000

# Benchmark
bench_sample = benchmark_pred.sample(min(sample_size, len(benchmark_pred)), random_state=42)
axes[0].scatter(bench_sample['y_pred'], bench_sample['y_true'], alpha=0.3, s=10)
axes[0].plot([-0.5, 0.5], [-0.5, 0.5], 'r--', alpha=0.5, label='Perfect prediction')
axes[0].set_title('OLS-3 Benchmark', fontweight='bold')
axes[0].set_xlabel('Predicted Return')
axes[0].set_ylabel('Actual Return')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# GBRT
gbrt_sample = gbrt_pred.sample(min(sample_size, len(gbrt_pred)), random_state=42)
axes[1].scatter(gbrt_sample['y_pred'], gbrt_sample['y_true'], alpha=0.3, s=10, color='orange')
axes[1].plot([-0.5, 0.5], [-0.5, 0.5], 'r--', alpha=0.5, label='Perfect prediction')
axes[1].set_title('GBRT', fontweight='bold')
axes[1].set_xlabel('Predicted Return')
axes[1].set_ylabel('Actual Return')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Feature Importance Deep Dive

In [ ]:
# Load feature importance
importance_df = pd.read_csv(RESULTS_DIR / 'tables' / 'feature_importance_top50.csv')

print("Top 20 Most Important Features:")
print(importance_df.head(20).to_string(index=False))

In [ ]:
# Interactive feature importance plot
import plotly.express as px

top_30 = importance_df.head(30)

fig = px.bar(top_30, x='gain_pct', y='feature', orientation='h',
             title='Top 30 Features by Importance (Gain)',
             labels={'gain_pct': 'Importance (%)', 'feature': 'Feature'},
             hover_data=['split_pct'])

fig.update_layout(height=800, yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
# Load group importance
group_importance = pd.read_csv(RESULTS_DIR / 'tables' / 'feature_group_importance.csv', index_col=0)

print("Feature Group Importance:")
print(group_importance.to_string())

## 4. Portfolio Analysis

In [ ]:
# Create portfolios based on GBRT predictions
eval_df = gbrt_pred.copy()
eval_df['prediction'] = eval_df['y_pred']
eval_df['ret_excess'] = eval_df['y_true']

# Get market cap if available
mkt_cap_col = None
for col in ['mvel1', 'me', 'market_equity', 'size']:
    if col in test_df.columns:
        mkt_cap_col = col
        eval_df[col] = test_df[col]
        break

eval_df = eval_df.reset_index()

# Create decile portfolios (EW)
portfolios = create_portfolio_sorts(eval_df, n_portfolios=10)

print(f"Created {portfolios['portfolio'].nunique()} portfolios")
print(f"\nPortfolio statistics:")
print(portfolios.groupby('portfolio')['n_stocks'].mean())

In [ ]:
# Portfolio returns over time
portfolio_pivot = portfolios.pivot(index='date', columns='portfolio', values='return')

# Calculate cumulative returns
cumulative = (1 + portfolio_pivot).cumprod()

# Plot
fig, ax = plt.subplots(figsize=(14, 7))

for port in [1, 5, 10]:
    if port == 1:
        label = 'P1 (Lowest predictions)'
        style = {'linewidth': 2, 'linestyle': '--', 'alpha': 0.7}
    elif port == 5:
        label = 'P5 (Medium)'
        style = {'linewidth': 1.5, 'alpha': 0.6}
    else:
        label = 'P10 (Highest predictions)'
        style = {'linewidth': 2.5, 'alpha': 0.9}
    
    ax.plot(cumulative.index, cumulative[port], label=label, **style)

ax.set_title('Cumulative Returns by Portfolio Decile', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Return', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate portfolio statistics
portfolio_stats = []

for port in range(1, 11):
    port_returns = portfolio_pivot[port].dropna()
    
    stats = {
        'Portfolio': port,
        'Mean Return (%)': port_returns.mean() * 100,
        'Std Dev (%)': port_returns.std() * 100,
        'Sharpe Ratio': calculate_sharpe_ratio(port_returns.values),
        'Cumulative Return': (1 + port_returns).prod() - 1
    }
    portfolio_stats.append(stats)

portfolio_stats_df = pd.DataFrame(portfolio_stats)
print("\nPortfolio Performance Statistics:")
print(portfolio_stats_df.to_string(index=False))

In [ ]:
# Long-short portfolio analysis
ls_returns = calculate_long_short_returns(portfolios)

print("Long-Short Portfolio Statistics:")
print(f"  Mean monthly return: {ls_returns['long_short'].mean()*100:.4f}%")
print(f"  Annualized return: {ls_returns['long_short'].mean()*12*100:.2f}%")
print(f"  Volatility (ann.): {ls_returns['long_short'].std()*np.sqrt(12)*100:.2f}%")
print(f"  Sharpe ratio: {calculate_sharpe_ratio(ls_returns['long_short'].values):.4f}")
print(f"  Best month: {ls_returns['long_short'].max()*100:.2f}%")
print(f"  Worst month: {ls_returns['long_short'].min()*100:.2f}%")

# Plot
cumulative_ls = (1 + ls_returns['long_short']).cumprod()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Cumulative returns
axes[0].plot(cumulative_ls.index, cumulative_ls.values, linewidth=2, color='darkgreen')
axes[0].set_title('Long-Short Portfolio Cumulative Returns', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Cumulative Return', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(1, color='black', linestyle='--', alpha=0.5)

# Monthly returns
axes[1].bar(ls_returns.index, ls_returns['long_short'] * 100, alpha=0.7, color='darkgreen')
axes[1].set_title('Long-Short Monthly Returns', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Monthly Return (%)', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.8)

plt.tight_layout()
plt.show()

## 5. Custom Analysis

Use the cells below for your own exploration!

In [ ]:
# Your custom analysis here